In [53]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import *
import matplotlib as mpl

mpl.rcParams.update(mpl.rcParamsDefault)

plt.rc('font', family='serif')
plt.rc('font', size=12)
#plt.rc('text', usetex=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
filename = f"data/evaluation_heavyside03_noise_h07.csv"
method = "normal"
df = pd.read_csv(filename)
df_defect = df[df.iloc[:,1] == method+"_defected_data"]
df_typical = df[df.iloc[:,1] == method+"_typical_data"]


if(len(df_defect) > 0):
    FNR = len(df_defect[df_defect.iloc[:,0]==False])/len(df_defect)
else:
    FNR = "No Data"
if(len(df_typical) > 0):
    FPR = len(df_typical[df_typical.iloc[:,0]==True])/len(df_typical)
else:
    FPR = "No Data"

print(f"False Negative Rate {FNR} (type II error)")
print(f"False Positive Rate {FPR} (type I error)")


False Negative Rate 0.032 (type II error)
False Positive Rate 0.108 (type I error)


# ROC Curves

In [103]:
#filename = f"data/evaluation_heavyside03_noise_h07_roc.csv"
filename = f"data/evaluation_heavyside15_noise_h07_roc.csv"
#filename = f"data/evaluation_franke_roc.csv"
#filename = f"data/evaluation_franke_roc_02.csv"
#filename = f"data/evaluation_heavyside_roc.csv"
#filename = f"data/evaluation_linear_roc.csv"
plt.figure(dpi=600)
df = pd.read_csv(filename)
method_list = ["normal", "wild", "wild_extended_smoothed"] # , "wild_extended" "normal", "wild_extended_smoothed"
alpha = []
fpr_lists = []
tpr_lists = []
for method in method_list:
    df_defect = df[df["name"] == method+"_defected_data"]
    df_typical = df[df["name"] == method+"_typical_data"]

    FPR_list = [0]
    TPR_list = [0]
    if(df_defect.shape[0] > 0):
        for col in df_defect.columns:
            split_col = col.split("_")
            if(split_col[0] != "rejected"):
                continue
            TPR_list.append(1-len(df_defect[df_defect[col]=="False"])/len(df_defect))
            if(len(alpha) < df_defect.shape[1]-1):
                alpha.append(float(split_col[1]))
    else:
        raise ValueError("No defected data")

    if(df_typical.shape[0] > 0):
        for col in df_typical.columns:
            split_col = col.split("_")
            if(split_col[0] != "rejected"):
                continue
            FPR_list.append(len(df_typical[df_typical[col]=="True"])/len(df_typical))
    else:
        raise ValueError("No typical data")
    
    fpr_lists.append(FPR_list[1:])
    tpr_lists.append(TPR_list[1:])

    FPR_list.append(1)
    TPR_list.append(1)

    plt.plot(np.array(FPR_list)*100, np.array(TPR_list)*100)

plt.xlabel("False Positive Rate (%)")
plt.xticks(np.linspace(0, 100, 21, endpoint=True), rotation=90)
plt.ylabel("True Positive Rate (%)")
plt.title(r"ROC Curve for $\sigma_{\epsilon}(x) = 0.3 + 1.5 \cdot h(x-0.7)$")
plt.legend(method_list)
plt.show()

plt.figure(dpi=600)
plt.plot(np.array(alpha)*100, ((np.array(fpr_lists)-np.array(alpha))/np.array(alpha)).T*100)
#plt.plot(np.array(alpha)*100, np.array(fpr_lists).T*100)
plt.xticks(np.linspace(0, 100, 21, endpoint=True), rotation=90)
plt.xlabel(r"Desired $\alpha$ level (%)")
plt.ylabel(r"$\frac{FPR-\alpha}{\alpha}$ (%)")
plt.title(r"Relative $\alpha$ level error for $\sigma_{\epsilon}(x) = 0.3 + 1.5 \cdot h(x-0.7)$") # 0.3 + 1.5 \cdot h(x-0.7)
plt.legend(method_list)
plt.show()

plt.figure(dpi=600)
plt.plot(np.array(alpha)*100, np.array(tpr_lists).T*100)
plt.xticks(np.linspace(0, 100, 21, endpoint=True), rotation=90)
plt.xlabel(r"Desired $\alpha$ level (%)")
plt.ylabel(r"$TPR$ (%)")
plt.title(r"True positive rate over $\alpha$ for $\sigma_{\epsilon}(x) = 0.3 + 1.5 \cdot h(x-0.7)$") # 0.7 - 1.4 \cdot (x-0.5)^2
plt.legend(method_list)
plt.show()